####Módulos

In [3]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

###Processo KDD - _Knowledge Discovery in Databases_

Composto pelas etapas de:

*   Seleção
*   Pré-Processamento
*   Transformação
*   Mineração de dados
*   Interpretação dos resultados



In [61]:
class GetData():
    def __init__(self):
        self.__df = None
        
    def select_data(self):
        path_1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/" \
               "wine-quality/winequality-red.csv"

        
        path_2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/" \
               "wine-quality/winequality-white.csv"
        
        df1 = pd.read_csv(path_1, delimiter=";")
        df1["Tipo"] = "Tinto"
        
        df2 = pd.read_csv(path_2, delimiter=";")
        df2["Tipo"] = "Branco"

        df = pd.merge(df1,df2,how="outer")

        dict = {'fixed acidity':'Acidez fixa',
                'volatile acidity':'Acidez volátil',
                'citric acid':'Ácido cítrico',
                'residual sugar':'Açúcar residual',
                'chlorides':'Cloretos',
                'free sulfur dioxide':'Dióxido de enxofre livre',
                'total sulfur dioxide':'Dióxido de enxofre total',
                'density':'Densidade',
                'sulphates':'Sulfatos',
                'alcohol':'Álcool',
                'quality':'Qualidade'}

        df.rename(columns=dict, inplace=True)

        self.__df = df
           


    def get_df(self):
        return self.__df

In [62]:
def apply_stratified_holdout(df: pd.DataFrame) -> tuple:
    
    features = ['Acidez fixa',
                'Acidez volátil',
                'Ácido cítrico',
                'Açúcar residual',
                'Cloretos',
                'Dióxido de enxofre livre',
                'Dióxido de enxofre total',
                'Densidade',
                'pH',
                'Sulfatos',
                'Álcool',
                'Qualidade']

    target = 'Tipo'
    
    X = df.drop(target, axis=1)
    Y = df[target]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, 
                                                        test_size=0.3,
                                                        stratify=Y) 
                                                        
    
    train_test_set = x_train, x_test, y_train, y_test
        
    return train_test_set

data = GetData()
df = data.select_data()
df = data.get_df()

nn_architectures = [
                    
        {"max_iter" : 100,
         "hidden_layer_sizes" : (10),
         "solver" : "lbfgs",
         "activation" : "relu"},

        {"max_iter" : 300,
         "hidden_layer_sizes" : (10,10,10),
         "solver" : "adam",
         "activation" : "logistic"}

    ]   


def scaling(train, test):
       
    scaler = StandardScaler().fit(train)
    x_train_scaled = scaler.transform(train)
    x_test_scaled = scaler.transform(test)
    
    return x_train_scaled, x_test_scaled
    

def model_training(arch, x, y):
    
    mlp_clf = MLPClassifier(max_iter = arch["max_iter"], 
                            hidden_layer_sizes = arch["hidden_layer_sizes"],
                            solver = arch["solver"],
                            activation = arch["activation"])
    
    model = mlp_clf.fit(x, y)
    
    return model



def show_results():

    num_executions = 10
    bases = ["Base de treino", "Base de teste"]
    iterations = list(range(num_executions))

    architectures = ["Tipo 1", "Tipo 2"]
    measures_labels = ["Matriz de confusão",
                        "Sensibilidade",
                        "Especificidade",
                        "Confiabilidade positiva",
                        "Confiabilidade negativa",
                        "Acurácia total"
        ]

    rows_multi_index = pd.MultiIndex.from_product((bases, iterations), names=["Base", "Execução"])
    columns_multi_index = pd.MultiIndex.from_product((architectures, measures_labels), names=["Arquitetura", "Medida"])

    df_measures = pd.DataFrame(index=rows_multi_index, columns=columns_multi_index)
    
    for i in range(num_executions):
        for arch in nn_architectures:

            x_train, x_test, y_train, y_test = apply_stratified_holdout(df)
            x_train, x_test = scaling(x_train, x_test) 

            trained_model = model_training(arch, x_train, y_train)

            features_dict = {"Base de treino": x_train, 
                             "Base de teste": x_test}
            
            target_dict = {"Base de treino": y_train,
                           "Base de teste": y_test}

            for base in bases:
                y_pred = trained_model.predict(features_dict[base])
                
                confusion_matrix = metrics.confusion_matrix(target_dict[base], y_pred)
                true_negatives, false_positives, false_negatives, true_positives = confusion_matrix.ravel()
                
                sensitivity = true_positives / (true_positives + false_negatives)
                specificity = true_negatives / (true_negatives + false_positives)
                positive_predictive_value = true_positives / (true_positives + false_positives)
                negative_predictive_value = true_negatives / (true_negatives + false_negatives)
                accuracy = metrics.accuracy_score(target_dict[base], y_pred)
                
                measures = np.array([sensitivity, 
                                     specificity, 
                                     positive_predictive_value, 
                                     negative_predictive_value, 
                                     accuracy])
                
                df_measures.loc[(base, i), 
                               (f"Tipo {nn_architectures.index(arch) + 1}", 
                                "Matriz de confusão")] = str(np.matrix(confusion_matrix)).replace("\n", ",").replace("[ ", "[")
                
                df_measures.loc[(base, i), (f"Tipo {nn_architectures.index(arch) + 1}", measures_labels[1:])] = measures
                
            for label in measures_labels[1:]:
                df_measures.loc[:, (f"Tipo {nn_architectures.index(arch) + 1}", label)] = pd.to_numeric(
                    df_measures.loc[:, (f"Tipo {nn_architectures.index(arch) + 1}", label)]
                )
        
    return df_measures


results = show_results()      
results

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Arquitetura                                 Tipo 1                \
Medida                          Matriz de confusão Sensibilidade   
Base           Execução                                            
Base de treino 0         [[3427    1], [  0 1119]]      1.000000   
               1         [[3426    2], [  0 1119]]      1.000000   
               2         [[3426    2], [  0 1119]]      1.000000   
               3         [[3427    1], [  0 1119]]      1.000000   
               4         [[3426    2], [  2 1117]]      0.998213   
               5         [[3427    1], [  1 1118]]      0.999106   
               6         [[3428    0], [  2 1117]]      0.998213   
               7         [[3428    0], [  0 1119]]      1.000000   
               8         [[3427    1], [  0 1119]]      1.000000   
               9         [[3428    0], [  1 1118]]      0.999106   
Base de teste  0         [[1466    4], [  8  472]]      0.983333   
               1         [[1458   12], [  7  473]]      0.985417   
               2         [[1467    3], [  2  478]]      0.995833   
               3         [[1462    8], [  5  475]]      0.989583   
               4         [[1466    4], [  6  474]]      0.987500   
               5         [[1466    4], [  4  476]]      0.991667   
               6         [[1466    4], [  6  474]]      0.987500   
               7         [[1462    8], [  7  473]]      0.985417   
               8         [[1466    4], [  5  475]]      0.989583   
               9         [[1465    5], [ 10  470]]      0.979167   

Arquitetura                                                     \
Medida                  Especificidade Confiabilidade positiva   
Base           Execução                                          
Base de treino 0              0.999708                0.999107   
               1              0.999417                0.998216   
               2              0.999417                0.998216   
               3              0.999708                0.999107   
               4              0.999417                0.998213   
               5              0.999708                0.999106   
               6              1.000000                1.000000   
               7              1.000000                1.000000   
               8              0.999708                0.999107   
               9              1.000000                1.000000   
Base de teste  0              0.997279                0.991597   
               1              0.991837                0.975258   
               2              0.997959                0.993763   
               3              0.994558                0.983437   
               4              0.997279                0.991632   
               5              0.997279                0.991667   
               6              0.997279                0.991632   
               7              0.994558                0.983368   
               8              0.997279                0.991649   
               9              0.996599                0.989474   

Arquitetura                                                     \
Medida                  Confiabilidade negativa Acurácia total   
Base           Execução                                          
Base de treino 0                       1.000000       0.999780   
               1                       1.000000       0.999560   
               2                       1.000000       0.999560   
               3                       1.000000       0.999780   
               4                       0.999417       0.999120   
               5                       0.999708       0.999560   
               6                       0.999417       0.999560   
               7                       1.000000       1.000000   
               8                       1.000000       0.999780   
               9                       0.999708       0.999780   
Base de teste  0                       0.994573       0.99

In [51]:
def get_average_std():
    rows = ["Base de treino", "Base de teste"]
    columns = ["Tipo 1", "Tipo 2"]

    accuracy_mean_1_train = results.loc[(rows[0]), ("Tipo 1", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_1_train = results.loc[(rows[0]), ("Tipo 1", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_1_test = results.loc[(rows[1]), ("Tipo 1", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_1_test = results.loc[(rows[1]), ("Tipo 1", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_2_train = results.loc[(rows[0]), ("Tipo 2", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_2_train = results.loc[(rows[0]), ("Tipo 2", ["Acurácia total"])].std().to_numpy()

    accuracy_mean_2_test = results.loc[(rows[1]), ("Tipo 2", ["Acurácia total"])].mean().to_numpy()
    accuracy_std_2_test = results.loc[(rows[1]), ("Tipo 2", ["Acurácia total"])].std().to_numpy()

    cell_11 = f"{accuracy_mean_1_train * 100}% ± {accuracy_std_1_train * 100}%"
    cell_12 = f"{accuracy_mean_1_test * 100}% ± {accuracy_std_1_test * 100}%"
    cell_21 = f"{accuracy_mean_2_train* 100}% ± {accuracy_std_2_train * 100}%"
    cell_22 = f"{accuracy_mean_2_test* 100}% ± {accuracy_std_2_test * 100}%"

    acc_data = [[cell_11, cell_12],
               [cell_21, cell_22]]

    df_average_std = pd.DataFrame(acc_data,index=rows, columns=columns, dtype=np.object_)

    df_average_std.index.name = "Base"
    df_average_std.columns.name = "Arquitetura"

               
    return df_average_std

get_average_std()

Arquitetura,Tipo 1,Tipo 2
Base,,
Base de treino,[99.94721795]% ± [0.02781859]%,[99.47179487]% ± [0.16404914]%
Base de teste,[99.63932263]% ± [0.06150921]%,[99.56923077]% ± [0.11130818]%
